# Documentation

# Question 1 

I chose Simmulated Annealing as a stochastic optimiser for Task 3 

# Question 2 Why did you choose this optimiser?

The spring relies on multiple variables making the graphical image of what the data might look like very complex, this means 
we cannot just choose a simple algorithm to optimise the problem and a stochastic approach would be beneficial.
Simulated annealing allows distortion to be added to the data set to avoid getting stuck at a local minima,
this normally allows a good optimal to be found. Simulated annealing works better with continuous function approximation 
than GA algorithms. As f(x) is a continuous function this makes simulated annealing the better option that should be implemented
to solve this problem. [99 words]

# Question 3 What constraint handling method(s) are you using and why?

To handle the inequality constraints of g1 to g4 and to keep all the simulated annealing points within the feasible space I
have used death penalties. Any combinations of values (x1, x2, x3) that do not pass g1 to g4 are rejected as these would 
produce an f(x) outside of the feasible region and take up more computational power than necessary.

I have made use of adaptive penalities when checking that my random points generated for simulated annealing are within
the feasible region. In my "distort" method, infeasible parameters are rejected in suit of a more fitting value.


In [1]:
import numpy as np
import math
import random
import matplotlib as mpl
import matplotlib.pyplot as plt 

In [2]:
arr_X = np.array([1, 2, 3])

# Task 1 implementation of f(x)
Question 1

: the diameter of the wire (x0), 
the mean of the diameter of coil(x1)
and the number of active coils (x2)

In [3]:
def f(x, fCount=[0]):
    fun = float(x[0] ** 2)*x[1]*(2 + x[2])
#     print(fun)
    fCount[0]+=1
    return [fun, fCount[0]]

def g1(x, g1Count=[0]):
    g1Count[0]+=1
    fun = float(1 - (((x[1]**3)*x[2])/(71785*(x[0]**4))))
    if ( fun <= 0) :
        return [True, g1Count[0]]
    else:
        return [False, g1Count[0]]
    
def g2(x, g2Count=[0]):
    fun = (   (((4*x[1]**2) - (x[0]*x[1]))/(12566*((x[1]*(x[0]**3))-(x[0]**4)))) + (1/(5108*(x[0]**2)))  - 1 )
    g2Count[0]+=1
    if fun <= 0:
        return[True, g2Count[0]]
    else:
        return[False, g2Count[0]]
    
def g3(x, g3Count=[0]):
    fun = (1 - (140.45*x[0])/((x[1]**2)*x[2]))
    g3Count[0]+=1
    if fun <= 0:   
        return[True, g3Count[0]]
    else:
        return[False, g3Count[0]]
    
def g4(x, g4Count=[0]):
    fun = (  ((x[0] + x[1])/1.5)-1  )
    g4Count[0]+=1
    if fun <= 0:   
        return[True, g4Count[0]]
    else:
        return[False, g4Count[0]]
    

In [4]:
print(f(arr_X))
print(f(arr_X))
print(f(arr_X))

print (g1(arr_X))
print (g1(arr_X))
print (g1(arr_X))

print (g2(arr_X))
print (g2(arr_X))
print (g2(arr_X))

print (g3(arr_X))
print (g3(arr_X))
print (g3(arr_X))

print (g4(arr_X))
print (g4(arr_X))
print (g4(arr_X))  

[10.0, 1]
[10.0, 2]
[10.0, 3]
[False, 1]
[False, 2]
[False, 3]
[True, 1]
[True, 2]
[True, 3]
[True, 1]
[True, 2]
[True, 3]
[False, 1]
[False, 2]
[False, 3]


# Task 2 - Random Search

In [5]:
wire_lower = 0.005 #lower bound for diameter of the wire
wire_upper = 2 #upper bound for diameter of the wire

coil_lower = 0.25 #lower bound for diameter of coil
coil_upper = 1.3 #upper bound for diameter of the coil

coil_num_lower = 2 #lower bound for number of active coils
coil_num_upper = 15 #upper bound for number of active coils

In [6]:
def generate_random_array():
    #generate random number 
    x0 = np.random.uniform(low=0.05, high=2) #hightest + 1
    x1 = np.random.uniform(low=0.25, high=1.3) #hightest + 1
    x2 = np.random.uniform(low=2, high=15) #hightest + 1
    
    return np.array([x0, x1, x2])
    

In [7]:
def is_feasible(array):
    feasible = False
    g1_arr=g1(array)
    g2_arr=g2(array)
    g3_arr=g3(array)
    g4_arr=g4(array)

#     print(g1_arr)
#     print(g2_arr)
#     print(g3_arr)
#     print(g4_arr)

    if g1_arr[0] and g2_arr[0] and g3_arr[0] and g4_arr[0] :
        feasible = True
    else:
        feasible = False
    return feasible
    

In [8]:
def random_search(best_fx, best_x):
    
    arr_x = generate_feasible_random_array()
    
    fx = f(arr_x)
    if fx < best_fx:
        best_x = np.array([arr_x[0], arr_x[1], arr_x[2]])
#             print("best replaced")
            
    return best_x

In [9]:
def generate_feasible_random_array():
    array = generate_random_array()
    feasible = is_feasible(array)
    while not feasible:
        array = generate_random_array()
        feasible = is_feasible(array)
    return array

In [10]:
best_x = generate_feasible_random_array()
best_fx = f(best_x)

for x in range(0, 3000):
    best_x = random_search(best_fx, best_x)
print(best_x)
print(best_fx)
#print(arr_X)

#myArray = np.array(arrX)
#print(myArray)

# test = np.array([1, 2, 3])
#print (g1(arr_X))
# # #def searching(arr_X, best_X, best_fx):



[0.07160637 0.9197573  4.06402388]
[0.05420753148898238, 4]


# Task 3 - Simulated Annealing

In [11]:
def distort(array):
    x0 = array[0]
    x1 = array[1]
    x2 = array[2]
    
    x0_new = 1000 #not feasible to begin with
    x1_new = 1000 #not feasible to begin with
    x2_new = 1000 #not feasible to begin with
    
    while not is_feasible(np.array([x0_new, x1, x2])):
        x0_new = np.random.normal(x0, 1)
#     print("i did go here")
    
    while not is_feasible(np.array([x0_new, x1_new, x2])):
        x1_new = np.random.normal(x1,1)
#     print("i did go here")
    
    while not is_feasible(np.array([x0_new, x1_new, x2_new])):
        x2_new = np.random.normal(x2,1)
#     print("i did go here")
   # print(str(x0_new) + " " + str(x1_new) + " " + str(x2_new))
        
    return [x0_new, x1_new, x2_new]

In [12]:
#test of distort method
start_values_array = generate_feasible_random_array()
print(start_values_array)
print(distort(start_values_array))

[0.06839918 0.75704601 9.70797181]
[0.0680443128683636, 0.8214926328747297, 10.074786362337004]


#to get minimum fx
#aka the energy
#when system goes through change of configuratioj we have to calculate delta e, change in energy

#how do we chnge te configuration - randomly change the configuration c, making a move

#temperature start with max temp and then slowly decreases to t min

#for every value of temperature 
    #compute energy 
    #change config and get energy of new config
    #calculate change in energy 
    #if change in e is -ve
        #accept the move
    #else if
        #the closer we get to the end of the cycle, the less likely we are to accept bad decisions
        #compute prob e^ change in e/temperature > rand(0,1)
        #accept the move
#step 1 random 

#step 2 more careful

In [13]:
start_values_array = generate_feasible_random_array()
print("lets start at random " + str(start_values_array))
best_fx = f(start_values_array)
print("best fx so far " + str(best_fx))

#for every value of temperature 
#do for number of iterations
temp_max = 3000
temp_min = 0
for t in range(temp_max, temp_min, -1):
#     print(t)
    
    #new config
#     print("finding neighboring set of values to compare")
    new_neighbours = distort(start_values_array)
#     print(new_neighbours)
    new_fx = f(new_neighbours)
    
    #calculae change in energy
#     print("finding delta e")
    delta_fx = new_fx[0] - best_fx[0]
#     print(str(new_fx) + " " + str(best_fx))
    
    if delta_fx < 0:
        print("new delta e in the house by merit " + str(t) +" " + str(new_fx))
        best_fx = new_fx
        #print("I was bested")
    elif (math.exp(delta_fx/t) > random.uniform(1, 1.05)):
        print("new delta e in the house by probability" + str(t) +" " + str(new_fx))
        best_fx = new_fx
        #print(math.exp(delta_fx/t))
#     print(math.exp(delta_fx/t))
print("finished")
    

lets start at random [ 0.11685528  1.0912085  11.53944481]
best fx so far [0.20174615349143404, 3005]
new delta e in the house by merit 3000 [0.12828111824271124, 3006]
new delta e in the house by merit 2999 [0.12327827938003585, 3007]
new delta e in the house by merit 2991 [0.11323833736038758, 3015]
new delta e in the house by merit 2868 [0.10929608000374558, 3138]
new delta e in the house by merit 2744 [0.09913069848909492, 3262]
new delta e in the house by probability1803 [0.17068602325001245, 4203]
new delta e in the house by merit 1802 [0.13202705504116297, 4204]
new delta e in the house by merit 1801 [0.12802552083183008, 4205]
new delta e in the house by merit 1798 [0.12743770920051384, 4208]
new delta e in the house by merit 1793 [0.12300358333500606, 4213]
new delta e in the house by merit 1765 [0.12074525697879336, 4241]
new delta e in the house by merit 1758 [0.11413505053573386, 4248]
new delta e in the house by merit 1676 [0.11125076020496237, 4330]
new delta e in the hou

# Task 4 - Repeations

In [15]:
#task 4

#random search
best_x = generate_feasible_random_array()
best_fx = f(best_x)

for x in range(0, 21):
    
    best_x = random_search(best_fx, best_x)
print(best_x)
print(best_fx)


[0.06933534 0.74116486 8.46768548]
[0.09199194951553159, 6006]


In [18]:
#boxplot
import numpy as np
import matplotlib.pyplot as plt

# Fixing random state for reproducibility
np.random.seed(19680801)

# fake up some data
spread = np.random.rand(50) * 100
center = np.ones(25) * 50
flier_high = np.random.rand(10) * 100 + 100
flier_low = np.random.rand(10) * -100
data = np.concatenate((spread, center, flier_high, flier_low))

fig1, ax1 = plt.subplots()
ax1.set_title('Basic Plot')
ax1.boxplot(data)

{'whiskers': [<matplotlib.lines.Line2D at 0x26a4e42d880>,
 'caps': [<matplotlib.lines.Line2D at 0x26a4e42df40>,
 'boxes': [<matplotlib.lines.Line2D at 0x26a4e42d520>],
 'medians': [<matplotlib.lines.Line2D at 0x26a4e439640>],
 'fliers': [<matplotlib.lines.Line2D at 0x26a4e4399a0>],
 'means': []}

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import tkinter

%matplotlib notebook

mpl.use('TkAgg')
mu = 0
std = 1
x = np.linspace(start=-4, stop=4, num=100)
y = stats.norm.pdf(x, mu, std) 
plt.plot(x, y)
plt.show()

NameError: name 'mpl' is not defined